In [2]:
from udl import udl
from pyspark.sql.functions import *
from pyspark import Row

# udlVar = udl.UDL(spark,'/data')
anonymise_udf = udf(udl.UDL.anonymise)
spark.udf.register("anonymise_udf", anonymise_udf)


print (udl.UDL.anonymise('name', 'Leo'))
print (udl.UDL.anonymise('name', 'Leo Martins'))
print (udl.UDL.anonymise('email', 'lmartins@pontusvision.com'))

print (udl.UDL.anonymise('name', 'Lua'))
print (udl.UDL.anonymise('name', 'Leo Martins'))
print (udl.UDL.anonymise('email', 'lmartins@pontusvision.com'))


mylist = [
  {"type_activity_id":1,"type_activity_name":"leo@example.net"},
  {"type_activity_id":2,"type_activity_name":"neeraj@email.com"},
  {"type_activity_id":3,"type_activity_name":"neeraj@email.com"},
  {"type_activity_id":4,"type_activity_name":"rachel@email.com"},
  {"type_activity_id":5,"type_activity_name":"rachel@email.com"}
]

df = spark.createDataFrame(Row(**x) for x in mylist)
df.show()

#### NOTE: WE MUST COALESCE to 1 to anonymise consistently; otherwise, we end up with multiple dictionary entries spread around.
###
df2 = df.coalesce(1).withColumn("type_activity_name_masked", anonymise_udf(lit('address'),'type_activity_name'))
df2.show()


### IF we do not coalesce, then we get repeated entries:
df3 = df.withColumn("type_activity_name2", anonymise_udf(lit('email'),('type_activity_name')))
df3.show()


24/06/26 17:50:52 WARN SimpleFunctionRegistry: The function anonymise_udf replaced a previously registered function.


'anonymising $name on Leo becomes Jason Brown'

Jason Brown


'anonymising $name on Leo Martins becomes Jacob Stein'

Jacob Stein


...

anonymising $address on leo@example.net becomes 25102 Tom Loop Apt. 667+ 1) / 1]
South Adam, ND 20695
anonymising $address on neeraj@email.com becomes 385 Day Vista Suite 343
Rogerville, CA 44492
anonymising $address on neeraj@email.com becomes 385 Day Vista Suite 343
Rogerville, CA 44492
anonymising $address on rachel@email.com becomes 53146 Jennings Shoals
Mathisport, MH 63759
anonymising $address on rachel@email.com becomes 53146 Jennings Shoals
Mathisport, MH 63759


+----------------+------------------+-------------------------+
|type_activity_id|type_activity_name|type_activity_name_masked|
+----------------+------------------+-------------------------+
|               1|   leo@example.net|     25102 Tom Loop Ap...|
|               2|  neeraj@email.com|     385 Day Vista Sui...|
|               3|  neeraj@email.com|     385 Day Vista Sui...|
|               4|  rachel@email.com|     53146 Jennings Sh...|
|               5|  rachel@email.com|     53146 Jennings Sh...|
+----------------+------------------+-------------------------+



anonymising $email on leo@example.net becomes brian12@example.net
anonymising $email on neeraj@email.com becomes daniel02@example.net


+----------------+------------------+--------------------+
|type_activity_id|type_activity_name| type_activity_name2|
+----------------+------------------+--------------------+
|               1|   leo@example.net| brian12@example.net|
|               2|  neeraj@email.com| brian12@example.net|
|               3|  neeraj@email.com|daniel02@example.net|
|               4|  rachel@email.com| brian12@example.net|
|               5|  rachel@email.com| brian12@example.net|
+----------------+------------------+--------------------+



anonymising $email on rachel@email.com becomes brian12@example.net
anonymising $email on rachel@email.com becomes brian12@example.net
anonymising $email on neeraj@email.com becomes brian12@example.net
